In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
import bokeh.layouts
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool,
    BoxSelectTool,CustomJS, Slider
)

#### Load Data

In [2]:
import json

with open('NYC_boundries_cleaned.json') as data_file:
    
    nyc_neighborhoods = json.load(data_file)

In [5]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

# from bokeh.sampledata.us_counties import data as counties
# from bokeh.sampledata.unemployment import data as unemployment

palette.reverse()

# counties = {
#     code: county for code, county in counties.items() if county["state"] == "ny"
# }

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]
neighborhoods = list(nyc_neighborhoods.keys())
num_nodes = [len(neighborhood["Lat"]) for neighborhood in nyc_neighborhoods.values()]

# county_names = [county['name'] for county in counties.values()]
# county_rates = [unemployment[county_id] for county_id in counties]
color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=neighborhood_xs,
    y=neighborhood_ys,
    neighborhood=neighborhoods,
    node=num_nodes
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="NY Traffic", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'node', 'transform': color_mapper},
          fill_alpha=0.7, line_color="black", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@neighborhood"),
    ("Number of nodes", "@node%"),
    ("(Long, Lat)", "($x, $y)"),
]


checkbox_button_group = CheckboxButtonGroup(
        labels=["Uber", "Taxi"], active=[0, 1])
slider = Slider(start=0, end=23, step=1, title="Hour")
select1 = Select(title="Month:", value="Month", options=["April", "May", "June", "July", "August", "September"])
select2 = Select(title="Date:", value="Date", options=["1", "2", "3", "4", "5", "6", "7"])

show(bokeh.layouts.row(p, checkbox_button_group, widgetbox(slider, select1, select2, width=300)))